Imports

In [5]:
import folium
import pandas as pd
import os
import json
import matplotlib as mpl
import requests
import numpy as np
import shapely

Load raw data

In [6]:
path=os.getcwd()
path=path[:-9]
path=path+'data/raw/corona/se_corona.csv'
corona=pd.read_csv(path,low_memory=False,delimiter='\t')

In [21]:
#Loading in the metadata
path=os.getcwd() #'something\\somthingelse\\fyp2021\\notebooks'
path=path[:-9] #'something\\somthingelse\\fyp2021\\'
path=path+'data/raw/metadata/se_metadata.json' #'something/somthingelse/fyp2021/data\\raw\\corona\\se_corona.csv'
f=open(path,'r')
metadata=json.load(f)
f.close()

codes=[]
places=[]
for i in metadata['country_metadata']:
    codes.append(i['iso3166-2_code'])
    places.append(i['iso3166-2_name_en'])

newcol=[]
for i in corona['region_code']:
    ind=places.index(i)
    newcol.append(codes[ind])

corona['isocodes']=newcol

8001

In [38]:
#Creating a cases per region dictionary
casesperregion=dict()
for i in codes:
    casesperregion[i]=0
for i in codes:
    newcol=corona.loc[corona['isocodes']==i]['confirmed_addition']
    casesperregion[i]=sum(newcol)

In [42]:
coronabyiso=pd.DataFrame()
coronabyiso['isocodes']=casesperregion.keys()
coronabyiso['cases']=casesperregion.values()
newcol=[]
for i in metadata['country_metadata']:
    newcol.append(i['population'])
coronabyiso['population']=newcol


,isocodes,cases,population
0,SE-AB,149331,2377081
1,SE-AC,13205,271736
2,SE-BD,11640,250093
3,SE-C,23408,383713
4,SE-D,12638,297540
5,SE-E,21988,465495
6,SE-F,25319,363599
7,SE-G,13547,201469
8,SE-H,10801,245446
9,SE-I,2664,59686


In [43]:
path=os.getcwd()
path=path[:-9]
path=path+'data/raw/shapefiles/se.geojson'

with open(path) as f:
  shapefiles = json.load(f)

{'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}, 'features': [{'type': 'Feature', 'properties': {'iso_3166_2': 'SE-BD'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[19.931010376000046, 68.35019602500007], [19.971214640000085, 68.35128123000008], [20.010282023000087, 68.36389027900002], [20.245926554000107, 68.47737172400004], [19.96222294100005, 68.54119211900009], [20.151772095000126, 68.60682118700002], [20.236624796000086, 68.65906606100002], [20.338840779000066, 68.76546783500002], [20.35217329900007, 68.78567332000001], [20.35651412000007, 68.80458689400005], [20.34876265400007, 68.893108623], [20.34111454300009, 68.91005849300011], [20.102266073000095, 69.02214467400005], [20.36271529150011, 69.02925018350014], [20.62316451, 69.03635569300012], [20.675461059000042, 69.01816558800009], [20.795350383000084, 69.01124094699999], [20.86397668500004, 68.98628123000002], [20.911208944000066, 68.980700175], [20.93

Exploratoy weather data analysis

In [ ]:
path=os.getcwd()#Finding path on current machine 
path=path[:-9]
path=path+'data/raw/weather/weather.csv'
weather=pd.read_csv(path,low_memory=False,delimiter='\t')#Loading in the data
mask=[]#creating a mask with true and false values where location is sweden
for i in weather['iso3166-2']:
    if i[0:2]!='SE':
        mask.append(False)
    else:
        mask.append(True)
mask=pd.DataFrame(mask)
mask=weather['iso3166-2'][mask.any(True)]#making the mask values instead of boolean array
weather=weather.loc[weather['iso3166-2'].isin(mask)] #sorting weather by all values in mask making it sweden
weather['TemperatureAboveGround']=weather['TemperatureAboveGround']-273.15#converting to celcius 

In [ ]:
for i in weather:
    if type(weather[i][33])==type(weather['Surfacepressure'][33]):
        print('','The mean for the',i,'column =', weather[i].mean(),'\n',
        'The minimum for the',i,'column =', weather[i].min(),'\n',
        'The maximum for the',i,'column =', weather[i].max(),'\n',
        'The median for the',i,'column =', weather[i].median(),'\n','\n')

 The mean for the RelativeHumiditySurface column = 74.15066260559007 
 The minimum for the RelativeHumiditySurface column = 37.431395 
 The maximum for the RelativeHumiditySurface column = 98.465382 
 The median for the RelativeHumiditySurface column = 74.88477499999999 
 

 The mean for the SolarRadiation column = 7827743.200207749 
 The minimum for the SolarRadiation column = 88.472774 
 The maximum for the SolarRadiation column = 24888899.344615 
 The median for the SolarRadiation column = 7008161.107125 
 

 The mean for the Surfacepressure column = 2378167.941814284 
 The minimum for the Surfacepressure column = 2185675.899808 
 The maximum for the Surfacepressure column = 2494548.073748 
 The median for the Surfacepressure column = 2389239.6942290002 
 

 The mean for the TemperatureAboveGround column = 9.335915398723282 
 The minimum for the TemperatureAboveGround column = -16.471316 
 The maximum for the TemperatureAboveGround column = 25.07441700000004 
 The median for the Tem

In [46]:
def get_country_center(country):
    url = 'http://nominatim.openstreetmap.org/search?country='+country+'&format=json&polygon=0'
    response = requests.get(url).json()[0]
    return np.array([float(response.get(key)) for key in ['lat','lon']])
 
country_center = get_country_center("Sweden")

Folium map with GeoJson

In [47]:
m = folium.Map(location=country_center, zoom_start = 8)
m

In [13]:
#folium.GeoJson(path+'data/raw/shapefiles/se.geojson', name = "shapefiles").add_to(m)
folium.GeoJson('data/raw/shapefiles/se.geojson', name = "shapefiles").add_to(m)

folium.LayerControl().add_to(m)

m

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/shapefiles/se.geojson'

Geodata visualization

- Make choropeth maps describing the corona cases in the country
- Use the folium.Choropleth function to connect the geoJSON with your dataframe
- To do so, you need to identify, in the geoJSON, which attribute contains the iso3166-2 code (note: if it is a sub-attribute of an attribute, you can refer to it as "attribute.sub_attribute").
- Visualize the CASES variable with an appropriate fill_color scale (check folium's documentation to know the allowed values)
- Visualize the population variable
- Create a cases per population variable and visualize that as well

In [48]:
#mapping number of cases in a choropleth map
m_cases = m

folium.Choropleth(
    geo_data = shapefiles,
    name = 'cases',
    data = coronabyiso,
    columns = ["isocodes", "cases"],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legends_name = "Number of Cases",
).add_to(m_cases)

m_cases

In [50]:
#mapping population density in a choropleth map
m_pop = m

folium.Choropleth(
    geo_data = shapefiles,
    name = "population",
    data = coronabyiso,
    columns = ["isocodes", "population"],
    key_on = "properties.iso_3166_2",
    fill_color = "YlGn",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legends_name = "Population",
).add_to(m_pop)

m_pop

In [52]:
#mapping cases per capita in a choropleth map
coronabyiso['casespercapita']=coronabyiso['cases']/coronabyiso['population']

m_cases_pc = m

folium.Choropleth(
    geo_data = shapefiles,
    name = "cases",
    data = coronabyiso,
    columns = ["isocodes", "casespercapita"],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legends_name = "Number of Cases per Capita",
).add_to(m_cases_pc)

m_cases_pc

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=24195024-9b97-4fbb-b643-b1da4ede0b31' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>